In [2]:
import numpy as np
import pandas as pd

In [5]:
ratingData=pd.io.parsers.read_csv("ratings.dat",
           names=["user_id","movie_id","rating","time"],
           engine="python",delimiter="::")
movieData=pd.io.parsers.read_csv("movies.dat",
           names=["movie_id","title","genre"],
           engine="python",delimiter="::", encoding='latin-1')

In [6]:
print(ratingData)

         user_id  movie_id  rating       time
0              1      1193       5  978300760
1              1       661       3  978302109
2              1       914       3  978301968
3              1      3408       4  978300275
4              1      2355       5  978824291
...          ...       ...     ...        ...
1000204     6040      1091       1  956716541
1000205     6040      1094       5  956704887
1000206     6040       562       5  956704746
1000207     6040      1096       4  956715648
1000208     6040      1097       4  956715569

[1000209 rows x 4 columns]


In [7]:
print(movieData)

      movie_id                               title  \
0            1                    Toy Story (1995)   
1            2                      Jumanji (1995)   
2            3             Grumpier Old Men (1995)   
3            4            Waiting to Exhale (1995)   
4            5  Father of the Bride Part II (1995)   
...        ...                                 ...   
3878      3948             Meet the Parents (2000)   
3879      3949          Requiem for a Dream (2000)   
3880      3950                    Tigerland (2000)   
3881      3951             Two Family House (2000)   
3882      3952               Contender, The (2000)   

                             genre  
0      Animation|Children's|Comedy  
1     Adventure|Children's|Fantasy  
2                   Comedy|Romance  
3                     Comedy|Drama  
4                           Comedy  
...                            ...  
3878                        Comedy  
3879                         Drama  
3880              

In [8]:
ratingMatrix = np.ndarray(
    shape=(np.max(ratingData.movie_id.values), np.max(ratingData.user_id.values)),
    dtype=np.uint8)
ratingMatrix[ratingData.movie_id.values-1, ratingData.user_id.values-1] = ratingData.rating.values
print(ratingMatrix)

[[  5  45 171 ...  39 205   3]
 [  0   0   0 ...  39 205  65]
 [  0   0   0 ...  41 205  65]
 ...
 [128 161  58 ... 123   0   0]
 [128  28  30 ... 123   0   0]
 [128 215  32 ... 123   0   0]]


In [9]:
normalizedMatrix = ratingMatrix - np.asarray([(np.mean(ratingMatrix, 1))]).T
print(normalizedMatrix)

[[-48.74155629  -8.74155629 117.25844371 ... -14.74155629 151.25844371
  -50.74155629]
 [-63.70612583 -63.70612583 -63.70612583 ... -24.70612583 141.29387417
    1.29387417]
 [-74.61854305 -74.61854305 -74.61854305 ... -33.61854305 130.38145695
   -9.61854305]
 ...
 [ 56.3852649   89.3852649  -13.6147351  ...  51.3852649  -71.6147351
  -71.6147351 ]
 [ 56.64006623 -43.35993377 -41.35993377 ...  51.64006623 -71.35993377
  -71.35993377]
 [ 60.20281457 147.20281457 -35.79718543 ...  55.20281457 -67.79718543
  -67.79718543]]


In [11]:
A = normalizedMatrix.T / np.sqrt(ratingMatrix.shape[0] - 1)
U, S, V = np.linalg.svd(A)

In [12]:
def similar(ratingData, movie_id, top_n):
    index = movie_id - 1 # Movie id starts from 1
    movie_row = ratingData[index, :]
    magnitude = np.sqrt(np.einsum('ij, ij -> i', ratingData, ratingData)) #Einstein summation |  traditional matrix multiplication and is equivalent to np.matmul(a,b)
    similarity = np.dot(movie_row, ratingData.T) / (magnitude[index] * magnitude)
    sort_indexes = np.argsort(-similarity) #Perform an indirect sort along the given axis (Last axis)
    return sort_indexes[:top_n]

In [14]:
k = 50
movie_id = 2
top_n = 10

sliced = V.T[:, :k] # representative data
indexes = similar(sliced, movie_id, top_n)

print('Recommendations for Movie {0}: \n'.format(
movieData[movieData.movie_id == movie_id].title.values[0]))
for id in indexes + 1:
    print(movieData[movieData.movie_id == id].title.values[0])

Recommendations for Movie Jumanji (1995): 

Jumanji (1995)
Indian in the Cupboard, The (1995)
Beauty and the Beast (1991)
Santa Clause, The (1994)
Snow White and the Seven Dwarfs (1937)
Dumbo (1941)
Free Willy (1993)
Aladdin and the King of Thieves (1996)
Jungle Book, The (1994)
Pete's Dragon (1977)
